Read in all unformatted files. Each line is a data entry

In [1]:
import pandas as pd
import numpy as np

english_exclusive_pure=[i.strip() for i in list(open("../../data_unformatted/english_exclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]
english_inclusive_pure=[i.strip() for i in list(open("../../data_unformatted/english_inclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]
french_exclusive_pure=[i.strip() for i in list(open("../../data_unformatted/french_exclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]
french_inclusive_pure=[i.strip() for i in list(open("../../data_unformatted/french_inclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]

if len(english_inclusive_pure) != len(english_exclusive_pure):
  raise Exception("uneven enlgish pairs")

if len(french_inclusive_pure) != len(french_exclusive_pure):
  raise Exception("uneven french pairs")

setup for zero-shot

In [2]:
both_exclusive_pure=english_exclusive_pure+french_exclusive_pure
both_inclusive_pure=english_inclusive_pure+french_inclusive_pure

# taskDescriptionOne="classify each of the sentences as inclusive or exclusive."
all_data=pd.DataFrame()

all_data["text"] = both_exclusive_pure+both_inclusive_pure
all_data["class"] = ['0']*len(both_exclusive_pure)+['1']*len(both_inclusive_pure)
print(all_data.head())

                                                text class
0  African immigrants are only able to seek unqua...     0
1           African Americans are always on welfare.     0
2  He looks Middle Eastern so he is probably here...     0
3  Oriental people built the transcontinental rai...     0
4            Everything is fair, I do not see color.     0


#### setup for few-shot  
- 4 random pairs
- 8 random pairs
- 4 inclusive and 4 exclusive. unrelated
- 8 inclusive and 8 exclusive. unrelated  
  
all are equal english and french.  
no sentence is reused. making datasets unique.

In [3]:
import random as ran

num_total_examples=len(english_inclusive_pure)+len(french_inclusive_pure)
both_example_range = range(num_total_examples)
single_example_range = range(len(english_exclusive_pure))

used_index = []

train_pairs_random_4=pd.DataFrame()
train_pairs_random_8=pd.DataFrame()
train_unrelated_random_8=pd.DataFrame()
train_unrelated_random_16=pd.DataFrame()

train_text_buffer=[]
train_class_buffer=[]

on_english=True
on_inclusive=True

train_pairs_random_4

In [4]:
train_text_buffer.clear()
train_class_buffer.clear()

four_ran_pairs=ran.sample(single_example_range, 4)
print(four_ran_pairs)

for i in four_ran_pairs:
  if on_english:
    train_text_buffer.append(english_inclusive_pure[i])
    train_class_buffer.append(1)
    train_text_buffer.append(english_exclusive_pure[i])
    train_class_buffer.append(0)
  else:
    train_text_buffer.append(french_inclusive_pure[i])
    train_class_buffer.append(1)
    train_text_buffer.append(french_exclusive_pure[i])
    train_class_buffer.append(0)

  on_english=not on_english

train_pairs_random_4["text"]=train_text_buffer
train_pairs_random_4["class"]=train_class_buffer

used_index += four_ran_pairs
train_pairs_random_4

[16, 59, 48, 1]


,text,class
0,The African continent is home to diverse cultu...,1
1,The African continent is nothing but war and p...,0
2,"La culture française est connue pour sa mode, ...",1
3,Les Français ne sont que superficialité et mod...,0
4,Metis people have a unique culture and history...,1
5,Metis people are just mixed race individuals w...,0
6,Les Afro-Américains dans les États dépendent d...,1
7,Les Afro-Américains sont toujours des assistés...,0


train_random_pairs_8

In [5]:
train_text_buffer.clear()
train_class_buffer.clear()

eight_ran_pairs=ran.sample(list(set(single_example_range)-set(four_ran_pairs)), 8)
print(eight_ran_pairs)

for i in eight_ran_pairs:
  if on_english:
    train_text_buffer.append(english_inclusive_pure[i])
    train_class_buffer.append(1)
    train_text_buffer.append(english_exclusive_pure[i])
    train_class_buffer.append(0)
  else:
    train_text_buffer.append(french_inclusive_pure[i])
    train_class_buffer.append(1)
    train_text_buffer.append(french_exclusive_pure[i])
    train_class_buffer.append(0)

  on_english=not on_english

train_pairs_random_8["text"]=train_text_buffer
train_pairs_random_8["class"]=train_class_buffer

used_index += eight_ran_pairs
train_pairs_random_8

[20, 61, 19, 29, 3, 41, 11, 82]


,text,class
0,Not all people from the Middle East are hostil...,1
1,All people from the Middle East are hostile.,0
2,"La culture espagnole est diverse, connue pour ...",1
3,Les Espagnols sont tous paresseux et n'ont pas...,0
4,"The black community, like all communities, has...",1
5,The black community is always looking for a ha...,0
6,Les gens des Caraïbes ont des origines diverse...,1
7,Les gens des Caraïbes sont des trafiquants de ...,0
8,Many Asian American people contributed to buil...,1
9,Oriental people built the transcontinental rai...,0


4 inclusive and 4 exclusive. unrelated

In [6]:
train_text_buffer.clear()
train_class_buffer.clear()

eight_ran_unrelated=ran.sample(list(set(single_example_range)-set(used_index)), 8)

english_slice=eight_ran_unrelated[0 : (len(eight_ran_unrelated)//2)]
french_slice=eight_ran_unrelated[(len(eight_ran_unrelated)//2) : len(eight_ran_unrelated)]

print(eight_ran_unrelated, english_slice, french_slice)

for i in english_slice:
  if on_inclusive:
    train_text_buffer.append(english_inclusive_pure[i])
    train_class_buffer.append(1)
  else:
    train_text_buffer.append(english_exclusive_pure[i])
    train_class_buffer.append(0)
  on_inclusive = not on_inclusive

for i in french_slice:
  if on_inclusive:
    train_text_buffer.append(french_inclusive_pure[i])
    train_class_buffer.append(1)
  else:
    train_text_buffer.append(french_exclusive_pure[i])
    train_class_buffer.append(0)
  on_inclusive = not on_inclusive

train_unrelated_random_8["text"]=train_text_buffer
train_unrelated_random_8["class"]=train_class_buffer

used_index += eight_ran_unrelated
train_unrelated_random_8

[14, 18, 22, 4, 83, 56, 15, 72] [14, 18, 22, 4] [83, 56, 15, 72]


,text,class
0,"Native American cultures are rich and diverse,...",1
1,All Hispanic people are illegal immigrants.,0
2,"African culture is diverse and rich, with a wi...",1
3,"Everything is fair, I do not see color.",0
4,Les Russes sont divers et viennent de milieux ...,1
5,"Les Texans sont fiers, beauf et armés.",0
6,Tous les musulmans ne sont pas des terroristes...,1
7,La culture singapourienne ne s'intéresse qu'à ...,0


8 inclusive and 8 exclusive. unrelated 

In [7]:
train_text_buffer.clear()
train_class_buffer.clear()

sixteen_ran_unrelated=ran.sample(list(set(single_example_range)-set(used_index)), 16)

english_slice=sixteen_ran_unrelated[0 : (len(sixteen_ran_unrelated)//2)]
french_slice=sixteen_ran_unrelated[(len(sixteen_ran_unrelated)//2) : len(sixteen_ran_unrelated)]

print(sixteen_ran_unrelated, english_slice, french_slice)

for i in english_slice:
  if on_inclusive:
    train_text_buffer.append(english_inclusive_pure[i])
    train_class_buffer.append(1)
  else:
    train_text_buffer.append(english_exclusive_pure[i])
    train_class_buffer.append(0)
  on_inclusive = not on_inclusive

for i in french_slice:
  if on_inclusive:
    train_text_buffer.append(french_inclusive_pure[i])
    train_class_buffer.append(1)
  else:
    train_text_buffer.append(french_exclusive_pure[i])
    train_class_buffer.append(0)
  on_inclusive = not on_inclusive

train_unrelated_random_16["text"]=train_text_buffer
train_unrelated_random_16["class"]=train_class_buffer

used_index += sixteen_ran_unrelated
train_unrelated_random_16

[60, 23, 67, 24, 69, 76, 53, 17, 75, 64, 34, 63, 40, 38, 13, 36] [60, 23, 67, 24, 69, 76, 53, 17] [75, 64, 34, 63, 40, 38, 13, 36]


,text,class
0,British people have a diverse range of backgro...,1
1,Arab culture is all about oppression of women.,0
2,People from West Bengal have a rich history an...,1
3,Asian culture is just about obedience and subm...,0
4,"Chinese culture has a deep-rooted history, ful...",1
5,Arabic people are extremists.,0
6,"Colombia has a complex history and society, an...",1
7,The Jewish community only cares about money.,0
8,L'Iran est un pays diversifié avec une histoir...,1
9,Les Maharashtriens ne s'intéressent qu'à l'arg...,0


In [8]:
#ensure all are unique and length = 40.
if (len(used_index)==36 and (len(set(used_index))==len(used_index))):
  print("Length==36 and all entires are unique")
else:
  print("its not good")

used_index_84 = [x+84 for x in used_index]
used_index_168 = [x+84 for x in used_index_84]
used_index_252 = [x+84 for x in used_index_168]
compiled_used_index = used_index+used_index_84+used_index_168+used_index_252

test_set = all_data.take(list(set(range(len(all_data)))-set(compiled_used_index)))
print(f"zero_shot shape: {all_data.shape}\nfew_shot_test shape: {test_set.shape}")

Length==36 and all entires are unique
zero_shot shape: (336, 2)
few_shot_test shape: (192, 2)


*^to maintin complete unrelatedness from train to test for few-shot,  
any language or opposite part of pair is not included in test set.  
as a result, every used_index from any of the four files is removed from every file in the few-shot test set*

##### to csv files.

In [9]:
train_pairs_random_4.to_csv("../../data_ready/few_shot/train_pairs_random_4.csv", index=False)
train_pairs_random_8.to_csv("../../data_ready/few_shot/train_pairs_random_8.csv", index=False)
train_unrelated_random_8.to_csv("../../data_ready/few_shot/train_unrelated_random_8.csv", index=False)
train_unrelated_random_16.to_csv("../../data_ready/few_shot/train_unrelated_random_16.csv", index=False)
test_set.to_csv("../../data_ready/unused_pairs_for_test_data.csv", index=False)